# Data challenge

Your task is to build an algorithm for prohibitory traffic sign recognition. We have
created a training data set (20 images) that can be used for building and fine-tuning a model.
However, you are encouraged to collect more images, since it will allow you to improve your
model’s performance.

There are 6 traffic sign categories your model should be able to recognize:
* Category A - no right, left, or U-turn
* Category B - speed limit (regardless of the indicated value)
* Category C - road closed
* Category D - no entry
* Category E - no stopping, no parking
* Category F - other types of prohibitory traffic signs

## Not I'll try to implement the [YOLOv8][1] in KerasCV, an extension of Keras for computer vision

Since pytorch is still a bit new to me and the origninal docs kind of forced me to upload our traffic to the roboflow website where they augmented the data without any code. I felt like that ruined the purpose of the data challenge. So now I'll follow this tutorial:

https://keras.io/examples/vision/yolov8/

[1]: https://keras.io/api/keras_cv/models/backbones/yolo_v8/


In [11]:
!pip install kerascv

In [12]:
# Import packages
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import keras_cv
from keras_cv import bounding_box
from keras_cv import visualization

# Define hyperparameters
SPLIT_RATIO = 0.2
BATCH_SIZE = 4
LEARNING_RATE = 0.001
EPOCH = 5
GLOBAL_CLIPNORM = 10.0

# Load in the Traffic Sign Data

In [13]:
class_ids = [
    'no-right-left-or-u-turn',
    'speed-limit',
    'road-closed',
    'no-entry',
    'no-stopping-no-parking',
    'other'
]

class_mapping = dict(zip(range(len(class_ids)), class_ids))
print(class_mapping)

# Path to images
path_images = '/kaggle/input/traffic-signs-data-challenge/train'

# Get all JPEG image file paths in path_images and sort them
jpg_files = sorted(
    [
        os.path.join(path_images, file_name)
        for file_name in os.listdir(path_images)
        if file_name.endswith(".jpg")
    ]
)

{0: 'no-right-left-or-u-turn', 1: 'speed-limit', 2: 'road-closed', 3: 'no-entry', 4: 'no-stopping-no-parking', 5: 'other'}


In [23]:
# Read the annotation file inot a pandas dataframe
annot_file = '/kaggle/input/traffic-signs-data-challenge/train/_annotations.csv'
annot_df = pd.read_csv(annot_file)

# Initialize lists to store information
image_paths = []
bbox = []
classes = []


def parse_annotation_csv(row):
    # Get the path pf the image
    image_name = row['filename']
    image_path = os.path.join(path_images, image_name)
    
    # Get the bounding box of the traffic sign
    box = [row['xmin'], row['ymin'], row['xmax'], row['ymax']]
    
    # Get the class id of the traffic sign
    class_id = row['class']
    
    return image_path, box, class_id

# Process annotations for each row in the DataFrame
for index, row in annot_df.iterrows():
    image_path, box, class_id = parse_annotation_csv(row)
    
    image_paths.append(image_path)
    bbox.append(box)
    classes.append(class_id)